In [9]:
from os import walk

def get_layout_files(layout):
   filenames = next(walk(layout), (None, None, []))[2]  # [] if no file
   return filenames

def all_files(n_layouts=4):
   for layout_id in range(n_layouts):
      layout = f'layout{layout_id}'
      for file in get_layout_files(layout):
         yield f"{layout}/{file}", layout_id

In [11]:
import pandas as pd

dataset = []

for file,layout_id in all_files():
    df = pd.read_csv(file)
    break
df

,Text,Tag
0,AVIVA,B-SUPP_N
1,DOORS,I-SUPP_N
2,Factory,O
3,Address:-,O
4,GAT,O
...,...,...
390,noid,O
391,challenoid,O
392,1,O
393,Authorised,O


In [6]:
import json 

# Serializing json  
json_object = json.dumps({   
    "name": "ainsretairn",
    "layout":1,
    'ner_tag':[],
    'token':[]
},) 
print(json_object)



{"name": "ainsretairn", "layout": 1, "ner_tag": [], "token": []}
